# Train a CNN for corner finding

In [66]:
# Imports

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Convolution2D, MaxPooling2D, Flatten
import tensorflow as tf

import PIL
import os
import random
import shutil
import pandas as pd
import numpy as np
import json

In [8]:
# allocate images to training, validation and test sets

# define the path to the pics
pic_path = r'J:\Drago Guggiana Nilo\Prey_capture\Corner_pics\Video\pics'
# define the paths to the divided datasets
train_path = r'J:\Drago Guggiana Nilo\Prey_capture\Corner_pics\Video\train'
val_path = r'J:\Drago Guggiana Nilo\Prey_capture\Corner_pics\Video\val'
test_path = r'J:\Drago Guggiana Nilo\Prey_capture\Corner_pics\Video\test'


if os.path.isdir(train_path) is False:
    os.makedirs(train_path)
    os.makedirs(val_path)
    os.makedirs(test_path)
    
    for c in random.sample(os.listdir(pic_path), 35):
        shutil.move(os.path.join(pic_path, c), train_path)
    
    for c in random.sample(os.listdir(pic_path), 10):
        shutil.move(os.path.join(pic_path, c), val_path)
    
    for c in random.sample(os.listdir(pic_path), 5):
        shutil.move(os.path.join(pic_path, c), test_path)

In [63]:
# parse the annotations
# define the path to the annotations
annotation_path = r'J:\Drago Guggiana Nilo\Prey_capture\Corner_pics\Video\via_project_28Aug2020_14h57m_csv.csv'

# read them as a dataframe
annotations = pd.read_csv(annotation_path)

# split them based on the data splits
# get a list of the train samples
train_list = os.listdir(train_path)

# allocate memory for the labels
# train_label = [el['filename'] for el in annotations.iterrows()]# if el['filename'] in train_list]
train_labels = []

annotated_names = annotations['filename'].to_numpy()

# get the unique labels
unique_labels = np.unique(annotations['region_attributes'])
# remove the empty
unique_labels = np.array([el for el in unique_labels if el != '{}'])
print(unique_labels)
for el in train_list:
    
    # get the current labels
    current_labels = annotations.loc[annotated_names==el,'region_attributes'].to_numpy()
    
    # if the file is not on the list, skip
    if len(current_labels) == 0:
        continue
    
    # allocate memory for the current coordinates
    current_coordinates = []
                
    # get the labels for this pic
    for label in unique_labels:
        
        # if it's in the list for this file
        if label in current_labels:
            
            # get the entry with the coordinates
            current_entry = annotations.loc[(annotated_names==el),:]
            current_entry = current_entry.loc[current_labels==label,'region_shape_attributes'].to_numpy()[0]
            current_entry = json.loads(current_entry.replace("'", "/"))
            
            # load the corresponding x and y values in the labels
            current_coordinates.append([current_entry['cx'], current_entry['cy']])
            
        else: 
            current_coordinates.append([np.nan, np.nan])


#     print(current_coordinates)
    # append to the main list
    train_labels.append(np.concatenate(current_coordinates))

# turn the list into an array
train_labels = np.array(train_labels)

print(train_labels.shape)

['{"Corner":"LL"}' '{"Corner":"LR"}' '{"Corner":"TL"}' '{"Corner":"TR"}']
(35, 8)


In [68]:
# preprocess the images

# allocate memory to store them
train_images = []


# train_images = tf.keras.preprocessing.image_dataset_from_directory(
#   train_path,
#   labels=train_labels,
#   image_size=(320, 256),
#   batch_size=5)

# # for all the images in the training list
# for image in train_list:
    
    # load the image
    current_image = PIL.Image.open(os.path.join(train_path,image))
    # resize
    current_image = tensorflow.image.resize(current_image, [320, 256])
    # scale
    current_image = tensorflow.image.per_image_standardization(current_image)
    # save
    train_images.append(current_image)

d:\programdata\miniconda3\envs\cellfinder\lib\site-packages\tensorflow\python\keras\preprocessing\image_dataset.py:142: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if labels != 'inferred':


ValueError: `labels` argument should be a list/tuple of integer labels, of the same size as the number of image files in the target directory. If you wish to infer the labels from the subdirectory names in the target directory, pass `labels="inferred"`. If you wish to get a dataset that only contains images (no labels), pass `label_mode=None`.

In [ ]:
# set up the validation data



In [2]:


# Activate GPU for this, otherwise the convnet will take forever to train with Theano.

# TODO: Make one run with very deep network (~10 layers).
filter_size = 3
pool_size = 2

# TODO: Maybe remove pooling bc it takes away the spatial information.

model = Sequential([
        Convolution2D(32, 6, 6, input_shape=X.shape[1:], dim_ordering='tf', activation='relu'), 
        MaxPooling2D(pool_size=(pool_size, pool_size)), 
        Convolution2D(64, filter_size, filter_size, dim_ordering='tf', activation='relu'), 
        MaxPooling2D(pool_size=(pool_size, pool_size)), 
        Convolution2D(128, filter_size, filter_size, dim_ordering='tf', activation='relu'), 
# #         MaxPooling2D(pool_size=(pool_size, pool_size)), 
        Convolution2D(128, filter_size, filter_size, dim_ordering='tf', activation='relu'), 
# #         MaxPooling2D(pool_size=(pool_size, pool_size)), 
        Flatten(), 
        Dropout(0.4), 
        Dense(256, activation='relu'), 
        Dropout(0.4), 
        Dense(y.shape[-1])
    ])

model.compile('adadelta', 'mse')

NameError: name 'X' is not defined

In [ ]:
# train the model
model.fit(x=train_images, y=train_labels, validation_data=validation_data, batch_size=5, epochs=30, shuffle=True verbose=2)

In [ ]:
# test the model